# 🔰 Getting started with cherwell_pydantic_api

## Step 1: create cherwell.env

The `cherwell.env` file contains the URL of your Cherwell API endpoint along with the credentials needed.

Use `cwcli` on the command line to create your `cherwell.env` file, and `cwcli check` to check connectivity.

## Step 2: Set up the business object filters

You can then proceed with this notebook. To get started, you can skip forward to the chapter **[Set up the business object filters](#bo_filter)**.

## (Boilerplate cells)

### Set up the environment and connect to the API

These cells set up the Jupyter notebook environment and the connection to the Cherwell API.

In [ ]:
# The working directory is expected to contain the cherwell.env configuration file.
# This implementation changes the working directory to the parent of the cherwell_pydantic_api package.
# Alter it if your setup is different.

import cherwell_pydantic_api
homedir = cherwell_pydantic_api.__path__[0] + '/../'
%cd $homedir

In [ ]:
# The Interactive module converts async/await calls to sync calls, so that you don't have to type await all the time.
# For this to work under Jupyter Notebook, the nest_asyncio module is required. (IPython on the console doesn't require this.)

import nest_asyncio
nest_asyncio.apply()

%gui asyncio

In [ ]:
# Import some useful classes into the namespace.

from cherwell_pydantic_api.bo.modelgen.repo import ModelRepo
from cherwell_pydantic_api.instance import Instance
from cherwell_pydantic_api.interactive import Interactive
from cherwell_pydantic_api.settings import Settings


# Set up the instance, repo and cw objects

instance = Instance.use()
print(f"base_url={instance.settings.base_url}")

cw = Interactive(waiter=get_ipython().loop_runner)

In [ ]:
# Authenticate to the API. This is the first actual HTTP callout. It might take a while if you are connecting to a sleeping test instance of Cherwell.

print("Authenticating...")
%time cw.authenticate()

### Set up the Collector

The `Collector` object collects Business Object schemas in preparation for Pydantic model generation.

This part of the notebook sets up widgets to make it easy to see the selected schemas that will be used.

In [ ]:
# Create the Collector object and async_wrap it in the cw Interactive instance.

from cherwell_pydantic_api.bo.modelgen.collector import Collector

cw.async_wrap(collector=Collector(instance, verbose=False))

In [ ]:
# Prepare some widgets for the datagrids below.

import ipywidgets
from ipyaggrid import Grid

status_output = ipywidgets.Output(layout={'border': '2px solid orange'})


#############################
## Set up bo_datagrid

datagrid_css = '''
.datagrid-verdict-true {
}
.datagrid-verdict-true::before {
    content: "";
    background-color: palegreen;
    position: absolute;
    right: 0;
    z-index: -10;
    width: 25%;
    height: 100%;
}
.datagrid-true {
    background-color: lightgreen;
}
.datagrid-false {
    background-color: lightpink;
}
'''

bo_column_defs = [
    {'field': 'name', 'sort': 'asc', 'type': 'style_verdict'},
    {'field': 'busobid', 'type': 'style_verdict'},
    {'field': 'verdict', 'type': 'bool'},
    {'field': 'bo_type', 'filter': True},
    {'field': 'num_fields'},
    {'field': 'displayName'},
    {'field': 'lookup', 'type': 'bool'},
    {'field': 'major', 'type': 'bool'},
    {'field': 'supporting', 'type': 'bool'},
    {'field': 'parent_name', 'filter': True},
]
bo_grid_options = {
    'columnDefs' : bo_column_defs,
    'rowSelection': 'single',
    'defaultColDef': {
        'sortable': True
    },
    'columnTypes': {
        'bool': {
            'filter': True,
            'floatingFilter': True,
            'cellClassRules': {
                'datagrid-true': 'x',
                'datagrid-false': '!x'
            }
        },
        'style_verdict': {
            'cellClassRules': {
                'datagrid-verdict-true': 'data.verdict'
            }
        }
    }
}
bo_datagrid = Grid(grid_data=[], grid_options=bo_grid_options,
                   columns_fit='auto', quick_filter=True, css_rules=datagrid_css,
                   export_mode='auto', export_to_df=False,
                   menu={'buttons': [
                    {'name': 'verdict true', 'action': '''gridOptions.api.getFilterInstance('verdict').setModel({values: ['true']}); gridOptions.api.onFilterChanged();'''},
                    {'name': 'clear verdict', 'action': '''gridOptions.api.destroyFilter('verdict')'''},
                   ]})


#############################
## Set up fld_datagrid

fld_column_defs = [
    {'field': 'name', 'sort': 'asc'},
    {'field': 'type', 'filter': True},
    {'field': 'short_field_id'},
    {'field': 'identifier', 'filter': True},
    {'field': 'fieldId'},
    {'field': 'type', 'filter': True},
    {'field': 'description', 'filter': True},
    {'field': 'category', 'filter': True},
    {'field': 'calculated', 'type': 'bool'},
    {'field': 'enabled', 'type': 'bool'},
    {'field': 'required', 'type': 'bool'},
]
fld_grid_options = {
    'columnDefs' : fld_column_defs,
    'defaultColDef': {
        'sortable': True
    },
    'columnTypes': {
        'bool': {
            'filter': True,
            'floatingFilter': True,
            'cellClassRules': {
                'datagrid-true': 'x',
                'datagrid-false': '!x'
            }
        },
    }
}

fld_datagrid = Grid(grid_data=[], grid_options=fld_grid_options,
                    columns_fit='auto', quick_filter=True, css_rules=datagrid_css)



##############################
# Set up interactivity

def datagrid_change(change):
    global cw, bo_datagrid, fld_datagrid, status_output
    if 'rows' not in bo_datagrid.grid_data_out:
        return None
    busobid = bo_datagrid.grid_data_out['rows'][0]['busobid']
    try:
        flds = cw.bo.get_schema(busobid).fieldDefinitions
        fld_datagrid.update_grid_data([vars(fld) for fld in flds])
    except Exception as e:
        fld_datagrid.update_grid_data([])
     

bo_datagrid.observe(datagrid_change, 'grid_data_out')


##############################################
# Run the collector and update the datagrid

def collect(include_filter, exclude_filter):
    global cw, bo_datagrid, status_output
    cw.collector.bo_include_filter = include_filter
    cw.collector.bo_exclude_filter = exclude_filter
    print("Collector loading data...")
    cw.collector.collect()
    print(f"OK, received {len(cw.collector.items)} summaries and {len([i for i in cw.collector.items if i.schema])} full schemas")
    bo_datagrid.update_grid_data([i.to_dict() for i in cw.collector.items])

collect(None, None);

<a id="bo_filter"></a>
## Set up the business object filters

In the following cell, edit the `bo_include_filter` and `bo_exclude_filter` regexes so that the table includes all the business objects that you wish to model.
They can be strings or re.Pattern objects.

In [ ]:
### EDIT THESE TWO VALUES ###

bo_include_filter = r'(?i)ticket$|config|changerequest$'
bo_exclude_filter = r''

############################################################################################
# If the cell below this one has been run, the table should update if you execute this cell.

status_output.clear_output()
with status_output:
    print(f"{bo_include_filter=!r}")
    print(f"{bo_exclude_filter=!r}")
    collect(bo_include_filter, bo_exclude_filter);


In [ ]:
# Display the collector output in a data grid. The collect() function updates it dynamically

ipywidgets.VBox([status_output,
                 ipywidgets.HTML("<h3>Business objects</h3>Business objects matching the filter will be indicated by the Verdict field being True. Click on the <b>verdict true</b> button to filter them. If the verdict is true, the <b>num_fields</b> column will show the number of fields, and you can select the row to examine each field below."),
                 bo_datagrid,
                 ipywidgets.HTML("<h3>Business object fields</h3>If a matching business object is selected above, the following table will display the fields."),
                 fld_datagrid])

## Step 3. Save the business object descriptions to your repository

Once you are happy with the business object filters, you can save the descriptions in your repository.

In [ ]:
# Print a list of the Business Objects that will be saved.

print("The following Business Objects will be saved to the repository.")
bo_list = [item.name for item in cw.collector.items if item.verdict]
bo_list.sort()
bo_list

In [ ]:
# Create the repository if it doesn't already exist

repo = ModelRepo(create=True)
print(f"Repository directory: {repo.directory} ; instance subpackage: {instance.settings.get_repo_subpackage()} ; instance branch: {instance.settings.get_repo_branch()}")

In [ ]:
# Save the business object registry

repo.save_instance(instance)

## Step 4. Generate Pydantic models and save them in your repository

You can now generate Pydantic models for the selected business objects.

In [ ]:
# Generate Pydantic models, and commit them in the repository.

cw.collector.save_models(repo)

You can also save the collector settings (the `bo_include_filter` and `bo_exclude_filter` values set previously) in the repository. This allows the models to be updated automatically with the `cwcli repo update` command.

In [ ]:
# Save the collector settings in the repository

cw.collector.save_settings(repo)